In [0]:
import tensorflow as tf
import kerastuner as kt
print(tf.__version__)
print(kt.__version__)

2.2.0-rc3
1.0.1


In [0]:
pip install keras-tuner

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=0fff5c80fa18af748df9f5f13200352c882dbc56d181129017d70dddc9c491f8
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=86754f0db96bac825358db0023686870e013923f045da1595eca287101cd6398
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
from tensorflow.keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [0]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers

In [0]:
# set random seed
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

In [0]:
# preprocessing - normalization
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [0]:
# model building
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))

In [0]:
# compile model using rmsprop
model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])

In [0]:
# model training
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=10)

Epoch 1/10
11/11 [==============================] - 0s 16ms/step - loss: 595.6030 - mse: 568.7303 - val_loss: 593.9056 - val_mse: 627.1449
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 585.4245 - mse: 555.4802 - val_loss: 581.5791 - val_mse: 614.4634
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 523.4369 - mse: 542.7706 - val_loss: 571.0361 - val_mse: 603.5865
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 528.6229 - mse: 532.2389 - val_loss: 559.8388 - val_mse: 592.0689
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 514.4418 - mse: 524.3663 - val_loss: 548.5232 - val_mse: 580.4174
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 506.9453 - mse: 510.7414 - val_loss: 536.4271 - val_mse: 567.9651
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 498.4555 - mse: 498.7845 - val_loss: 523.8084 - val_mse: 554.9809
Epoch 8/10
11/11 [===============

In [0]:
# model evaluation
model.evaluate(x_test_scaled, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 542.0904 - mse: 488.3325


[542.0903930664062, 488.33251953125]

# Tuning with Keras Tuner

In [0]:
from kerastuner import HyperModel
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
        )
        
        return model

# Instantiate HyperModel

In [0]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Random Search

In [0]:
tuner_rs = kt.tuners.randomsearch.RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=10,
            executions_per_trial=2)

In [0]:
tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


In [0]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(x_test_scaled, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 63.2154 - mse: 55.0461


In [0]:
tuner_hb = kt.tuners.hyperband.Hyperband(
            hypermodel,
            max_epochs=5,
            objective='mse',
            seed=42,
            hyperband_iterations=2,
            overwrite=True
        )



In [0]:
tuner_hb.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)


INFO:tensorflow:Oracle triggered exit


In [0]:
best_model = tuner_hb.get_best_models(num_models=1)[0]
best_model.evaluate(x_test_scaled, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 335.7272 - mse: 294.7773


[335.72723388671875, 294.7772521972656]

# Bayesian Optimization

In [0]:
tuner_bo = kt.tuners.bayesian.BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2,
            overwrite=True
        )
tuner_bo.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)
best_model = tuner_bo.get_best_models(num_models=1)[0]
best_model.evaluate(x_test_scaled, y_test)

INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 2ms/step - loss: 52.4451 - mse: 45.4841


[52.445106506347656, 45.484092712402344]

In [37]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 48)                672       
_________________________________________________________________
dense_1 (Dense)              (None, 48)                2352      
_________________________________________________________________
dropout (Dropout)            (None, 48)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 49        
Total params: 3,073
Trainable params: 3,073
Non-trainable params: 0
_________________________________________________________________


https://towardsdatascience.com/hyperparameter-tuning-with-keras-tuner-283474fbfbe